In [ ]:
from google.colab import files
files.upload()

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [6]:
#Load the Dataset
data = pd.read_csv('Sales.csv')

In [ ]:
print(data.head())

In [ ]:
print(data.describe())

In [9]:
X = data[['TV', 'Radio', 'Newspaper']]
y = data['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [11]:
y_pred = model.predict(X_test)

In [12]:
# Calculate metrics to evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2): {r2}")

In [ ]:
# Visualize the predictions
plt.scatter(X_test['TV'], y_test, label='Actual Sales')
plt.scatter(X_test['TV'], y_pred, label='Predicted Sales', color='red')
plt.xlabel('TV Advertising Spending')
plt.ylabel('Sales')
plt.legend()
plt.title('Future Sales Prediction')
plt.show()

#Using Facebook Prophet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet

# Load the dataset
data = pd.read_csv('Sales.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Prepare the data for Prophet
df = data[['Date', 'Sales']]
df = df.rename(columns={'Date': 'ds', 'Sales': 'y'})

# Initialize and fit the Prophet model
model = Prophet()
model.fit(df)

# Create a future dataframe for forecasting
future = model.make_future_dataframe(periods=365)  # Adjust the number of future periods as needed

# Make predictions
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)
plt.title('Sales Forecast with Prophet')
plt.show()


#Using LSTM Networks

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('Sales.csv')
sales_data = data[['Date', 'Sales']].copy()
sales_data['Date'] = pd.to_datetime(sales_data['Date'])
sales_data.set_index('Date', inplace=True)

# Prepare the data
scaler = MinMaxScaler()
sales_data['Sales'] = scaler.fit_transform(sales_data['Sales'].values.reshape(-1, 1))
sales_data['Date'] = sales_data.index

# Split data into training and testing
train_size = int(len(sales_data) * 0.8)
train_data, test_data = sales_data[:train_size], sales_data[train_size:]

# Create sequences for LSTM
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i:i+sequence_length]
        target = data[i+sequence_length:i+sequence_length+1]
        sequences.append((sequence, target))
    return sequences

sequence_length = 10  # Adjust as needed
train_sequences = create_sequences(train_data['Sales'], sequence_length)
test_sequences = create_sequences(test_data['Sales'], sequence_length)

# Define an LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Training the LSTM model
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loader = torch.utils.data.DataLoader(train_sequences, batch_size=64, shuffle=True)

num_epochs = 100

for epoch in range(num_epochs):
    for sequence, target in train_loader:
        optimizer.zero_grad()
        sequence = sequence.view(-1, sequence_length, 1)
        outputs = model(sequence)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

# Make predictions on the test set
test_loader = torch.utils.data.DataLoader(test_sequences, batch_size=1, shuffle=False)
predictions = []

model.eval()
with torch.no_grad():
    for sequence, _ in test_loader:
        sequence = sequence.view(-1, sequence_length, 1)
        prediction = model(sequence)
        predictions.append(prediction.item())

# Inverse scaling for predictions
predicted_sales = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Plot the predictions
plt.plot(test_data.index[sequence_length:], test_data['Sales'].values[sequence_length:], label='Actual Sales')
plt.plot(test_data.index[sequence_length:], predicted_sales, label='Predicted Sales', color='red')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.title('Sales Forecast with LSTM')
plt.show()
